In [97]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

In [98]:
from tqdm.notebook import tqdm
from pprint import pprint
from statsmodels.stats.weightstats import ztest

In [99]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import bs4
import re
import math
import ast

In [156]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [131]:
# для красивых графиков

%config InlineBackend.figure_format = 'retina'

sns.set(style='darkgrid', palette='deep')

plt.rcParams['font.size'] = 16
plt.rcParams['savefig.format'] = 'pdf'

In [132]:
df = pd.read_csv("eda_data.csv")
df

,artist,country,listeners_lastfm,scrobbles_lastfm,likes,tracks,albums,genres,years_active
0,Coldplay,United Kingdom,5381567.0,360111850.0,1305445,186,60,['rock'],26
1,Radiohead,United Kingdom,4732528.0,499548797.0,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0,2155459,266,32,['rock'],41
3,Rihanna,United States,4558193.0,199248986.0,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,United States,4517997.0,199507511.0,5278564,396,37,['foreignrap'],35
...,...,...,...,...,...,...,...,...,...
3834,Lyn Collins,United States,189533.0,845689.0,109,43,4,"['rnb', 'electronics']",43
3835,Saigon,United States,189479.0,1939393.0,82,298,42,"['rap', 'pop', 'african']",23
3836,Kwabs,United Kingdom,189460.0,1655444.0,1207967,28,8,['soul'],12
3837,Diablo Swing Orchestra,Sweden,189429.0,8016066.0,15532,60,5,['progmetal'],20


# Шаг 5. Создание новых признаков

Глобально мы хотим предсказывать переменную связанную с лайками, поэтому в создании признаков больше ее мы трогать не будем, чтобы не допустить утечки таргета.

Для начала в соответствии с графиками в EDA, добавим прологарифмированные переменные, чтобы улучшить их вид.

In [133]:
df['log_listeners'] = np.log(df.listeners_lastfm)
df['log_scrobbles'] = np.log(df.scrobbles_lastfm)
df['log_tracks'] = np.log(df.tracks)

Можно завести переменные, означающие отношение одной колонки к другой. Например это может быть отношение кол-во слушателей, кол-во скробблов, треков и альбомов к годам активности группы. А также отношение их логарифмов к годам. Это может быть важно, потому что группа может быть очень продуктивной в течение своего небольшого времени активности и написать мало альбомов, а может написать альбомов немногим большее кол-во, но за промежуток времени в 2 раза больший. Возможно первая группа будет более популярна, потому что музыка это не только про вдохновение, но и хороший менеджмент и энергию исполнителей.

In [134]:
df['listeners_per_years'] = df.listeners_lastfm / df.years_active
df['scrobbles_per_years'] = df.scrobbles_lastfm / df.years_active
df['tracks_per_years'] = df.tracks / df.years_active
df['albums_per_years'] = df.albums / df.years_active

df['log_listeners_per_years'] = df.log_listeners / df.years_active
df['log_scrobbles_per_years'] = df.log_scrobbles / df.years_active
df['log_tracks_per_years'] = df.log_tracks / df.years_active

Можно посмотреть на то, сколько треков содержится в среднем в одном альбоме.

In [135]:
df['tracks_per_album'] = df.tracks / df.albums

Кроме того, при парсинге данных мы не включили в данные год начала группы и год завершения выступлений. Давайте добавим их.

In [136]:
df_tags = pd.read_csv("big_lst_5000.csv", index_col=0)
df_tags

,artist_mb,parsed_data
0,Coldplay,"<td class=""infobox-data"">1997–present</td>"
1,Radiohead,"<td class=""infobox-data"">1985–present</td>"
2,Red Hot Chili Peppers,"<td class=""infobox-data"">1982<span style=""disp..."
3,Rihanna,"<td class=""infobox-data"">2003–present</td>"
4,Eminem,"<td class=""infobox-data"">1988–present<sup clas..."
...,...,...
4857,Kwabs,"<td class=""infobox-data"">2011–present</td>"
4858,Astral Projection,"<td class=""infobox-data"">1993–present</td>"
4859,Diablo Swing Orchestra,"<td class=""infobox-data"">2003–present</td>"
4860,Despised Icon,"<td class=""infobox-data"">2002–2010, 2014–prese..."


Воспользуемся кодом из предыдущих тетрадок.

In [137]:
def get_intervals(raw_intervals):
    intervals = []
    # split by '-'
    left_rights = re.findall(r'\S+–\S+', raw_intervals)
    for left_right in left_rights:
        parts = left_right.split('–')
        clean_parts = tuple(re.findall(r'(?:\d{4}|present)', part)[0] for part in parts)
        clean_parts_without_present = tuple(2023 if part == 'present' else int(part) for part in clean_parts)
        intervals.append(clean_parts_without_present)
    return intervals

def parse_years_text(s):
    # check hiaustes
    main_and_hiatuses = s.split('hiatus')
    main = main_and_hiatuses[0]
    hiastuses = ''
    if len(main_and_hiatuses) > 1:
        hiastuses = main_and_hiatuses[1]

    main_intervals = get_intervals(main)
    hiastuses_intervals = get_intervals(hiastuses)
        
    return main_intervals, hiastuses_intervals

def get_first_year(tag):
    try:
        years = bs4.BeautifulSoup(tag)
        intervals, _ = parse_years_text(years.text)
        return intervals[0][0]
    except:
        return None
    
def get_last_year(tag):
    try:
        years = bs4.BeautifulSoup(tag)
        intervals, _ = parse_years_text(years.text)
        return intervals[-1][1]
    except:
        return None



In [138]:
# Создадим новые столбцы
df_tags['first_year'] = df_tags.parsed_data.apply(lambda x: get_first_year(x))
df_tags['last_year'] = df_tags.parsed_data.apply(lambda x: get_last_year(x))

In [139]:
# Для последующего мерджа переименуем одну, и удалим вторую колонку
df_tags = df_tags.drop('parsed_data', axis=1)
df_tags = df_tags.rename({'artist_mb': 'artist'}, axis=1)
df_tags

,artist,first_year,last_year
0,Coldplay,1997.0,2023.0
1,Radiohead,1985.0,2023.0
2,Red Hot Chili Peppers,1982.0,2023.0
3,Rihanna,2003.0,2023.0
4,Eminem,1988.0,2023.0
...,...,...,...
4857,Kwabs,2011.0,2023.0
4858,Astral Projection,1993.0,2023.0
4859,Diablo Swing Orchestra,2003.0,2023.0
4860,Despised Icon,2002.0,2023.0


In [140]:
# смержим две таблички
df = pd.merge(df, df_tags, on='artist')

### Шаг 7: Машинное обучение

У нас датасет об исполнителях, в котором содержится различная информация: кол-во слушателей с сайта last.fm, кол-во треков, альбомов, различные другие признаки. Мы хотим предсказывать самое главное, что можно сказать об исполнителе - его успешность и популярность. И хотя и кол-во слушателей, и всего остального косвенно говорит о популярности, но все же кол-во лайков лучше описывает данное явление.

В качестве метрики будем использовать обычный MSE, так как у нас задача регрессии. 

Кол-во лайков это хорошо, но давайте предсказывть лучше логарифм от этой величины, потому что нам было бы важнее для задачи угадывать с порядком числа лайков. То есть по сути научиться выявлять степень популярности.


In [147]:
df['log_likes'] = np.log(df.likes)
df = df.drop('likes', axis=1)

Удаляем колонку с артистами, потому что все названия абсолютно различны.

In [142]:
df = df.drop('artist', axis=1)

In [143]:
# Превращаем строки содержащие список жарнов в питоновский список
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x))
df.genres

0                    [rock]
1                   [indie]
2                    [rock]
3         [pop, dance, rnb]
4              [foreignrap]
               ...         
3834     [rnb, electronics]
3835    [rap, pop, african]
3836                 [soul]
3837            [progmetal]
3838       [metalcoregenre]
Name: genres, Length: 3839, dtype: object

In [144]:
# Делаем One Hot Encoding для столбца со списком жанров
mlb = MultiLabelBinarizer()
df_mlb = pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                          columns=mlb.classes_,
                          index=df.index)
df_mlb = df_mlb.rename({'country': 'country_genre'}, axis=1)

df = df.join(df_mlb)
df.shape

(3839, 119)

In [145]:
# One Hot Encoding для country
df = pd.get_dummies(df, columns=['country'])
df.shape

(3839, 145)

Это модель-бейзлайн. Предсказываем mse.

In [148]:
X = df.drop('log_likes', axis=1)
y = df['log_likes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

2.6422045581239457

Обучим теперь модель с регуляризацией.

In [151]:
model = Ridge()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.88178e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


2.6262395817472792

Стало немного получше. Теперь подберем для него гиперпараметры.

In [155]:
model = Ridge()

param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.21146e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.91076e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.64808e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.21146e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Users\world\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAl

2.6262395817472792

Результаты не изменились.

Обучим градиентный бустинг с дефолтными параметрами.

In [158]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse


2.6937019021149378

Качество упало по сравнению с бейзлайном, давайте попробуем подобрать гиперпараметры.

In [163]:
model = RandomForestRegressor()

param_grid = {
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2)
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1; total time=   2.8s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1; total time=   2.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1; total time=   2.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2; total time=   1.7s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2; total time=   1.7s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2; total time=   1.6s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4; total time=   1.4s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4; total time=   1.4s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4; total time=   1.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1; total time=   0.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1; total time=   0.3s
[CV] END max_depth=None, m

2.7698851107153586

После перебора стало только хуже, видимо мы сильно ухудшили дефолтные параметры.

**Вывод**: самой оптимальной моделью оказалась Ridge-регрессия. Ее MSE ~ 2.62, то есть RMSE ~ 1.61. Получается мы ошибаемся с порядком в среднем на 1.61, то есть мы ошибаемся обычно в 1-2 знака числа. Это довольно много( Построить супер модель, которая предсказывае популярность (логарифм количества лайков) группы не получилось.

In [164]:
2.62 ** 0.5

1.6186414056238645